# Case study example -- vaccination related literature topic detection

In the first case study, we will explore, whether the existing COVID-19-related literature, that focuses on vaccines and treatment programs clusters in well defined topics that could help medical professionals further explore the existing studies. The case study is structured as follows.

1. Description of the preprocessing machinery and the request-ready API we designed around CORD-19 data set.
2. An example run, that is completely interactive
3. Discussion of results

In [1]:
## the code corresponding to c1.
import json
import requests
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet as wn
import nltk
from gensim import corpora
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
from gensim.models.ldamodel import LdaModel

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def prepare_text_for_lda(text):
    tokens = text.split(" ")
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens


[nltk_data] Downloading package wordnet to /home/blazs/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/blazs/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The above we first define some functions, that shall be used for the preprocessing purposes. Next, we simply quarry the API to fetch the documents, most related to the user specified keywords (interesting keywords).

In [2]:
interesting_keywords = ["covid-19","vaccines","treatment"]
json_data_all = []
for keyword in interesting_keywords:
    example_query = "http://covid19explorer.ijs.si/gp/api?keyword={}".format(keyword)
    response = requests.get(example_query)
    json_data = json.loads(response.text)
    json_data_all+=json_data

## get scores and titles
top_docs = []
for hit in json_data_all:
    title, abstract = hit['article_title'], hit['article_abstract']
    if len(abstract) > 30:
        top_docs.append(abstract)

## clean
clean_text = []
for el in top_docs:
    tokens = prepare_text_for_lda(el)
    clean_text.append(tokens)


Finally, let's use GENSIM's LDA package to detect some topics!

In [3]:
dictionary = corpora.Dictionary(clean_text)
corpus = [dictionary.doc2bow(text) for text in clean_text]

NUM_TOPICS = 10
ldamodel = LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)

topics = ldamodel.print_topics(num_words=5)

What are the representative words in these topics?

In [4]:
for topic in topics:
    print(topic)

(0, '0.010*"vaccine" + 0.008*"development" + 0.006*"12-year" + 0.006*"stage" + 0.006*"Harris"')
(1, '0.013*"treatment" + 0.007*"disease" + 0.006*"infection" + 0.006*"gulae" + 0.006*"treat"')
(2, '0.017*"vaccine" + 0.012*"technology" + 0.008*"disease" + 0.008*"vaccines." + 0.008*"infectious"')
(3, '0.021*"vaccine" + 0.017*"disease" + 0.012*"infectious" + 0.009*"treatment" + 0.007*"Symptoma"')
(4, '0.014*"COVID-19" + 0.013*"treatment" + 0.009*"control" + 0.007*"patient" + 0.006*"compare"')
(5, '0.021*"vaccine" + 0.014*"reaction" + 0.014*"staff" + 0.012*"allergic" + 0.010*"include"')
(6, '0.014*"treatment" + 0.009*"vaccination" + 0.006*"model" + 0.006*"equilibrium" + 0.006*"globally"')
(7, '0.012*"vaccine" + 0.008*"pathogen" + 0.006*"report" + 0.006*"infection" + 0.006*"vaccination"')
(8, '0.017*"vaccine" + 0.007*"treatment" + 0.005*"case" + 0.005*"edible" + 0.005*"vaccines,"')
(9, '0.033*"vaccine" + 0.013*"animal" + 0.011*"treatment" + 0.008*"safety" + 0.008*"patient"')


As we can see, there exist various sub-topics that might be of interest to a medical professional. To explore the abstracts in more detail, abstracts can be fetched too!

In [5]:
top_docs[-1]

'Currently, Severe Acute Respiratory Syndrome Coronavirus 2 (SARS-CoV-2, formerly known as 2019-nCoV, the causative pathogen of Coronavirus Disease 2019 (COVID-19)) has rapidly spread across China and around the world, causing an outbreak of acute infectious pneumonia. No specific anti-virus drugs or vaccines are available for the treatment of this sudden and lethal disease. The supportive care and non-specific treatment to ameliorate the symptoms of the patient are the only options currently. At the top of these conventional therapies, greater than 85% of SARS-CoV-2 infected patients in China are receiving Traditional Chinese Medicine (TCM) treatment. In this article, relevant published literatures are thoroughly reviewed and current applications of TCM in the treatment of COVID-19 patients are analyzed. Due to the homology in epidemiology, genomics, and pathogenesis of the SARS-CoV-2 and SARS-CoV, and the widely use of TCM in the treatment of SARS-CoV, the clinical evidence showing t

In [6]:
top_docs[3]

'COVID-19 is a novel coronavirus with an outbreak of unusual viral pneumonia in Wuhan, China, and then pandemic. Based on its phylogenetic relationships and genomic structures the COVID-19 belongs to genera Betacoronavirus. Human Betacoronaviruses (SARS-CoV-2, SARS-CoV, and MERS-CoV) have many similarities, but also have differences in their genomic and phenotypic structure that can influence their pathogenesis. COVID-19 is containing single-stranded (positive-sense) RNA associated with a nucleoprotein within a capsid comprised of matrix protein. A typical CoV contains at least six ORFs in its genome.'